# Гибридный RAG

### Импорты

In [18]:
import os

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

from pymilvus import MilvusClient
from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    WeightedRanker,
    connections,
)

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

from llamaapi import LlamaAPI

from tqdm import tqdm

from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain_milvus.utils.sparse import BM25SparseEmbedding

from langchain_text_splitters import RecursiveCharacterTextSplitter

import pickle as pkl

token = '' # Ваш токе

### Функции для запросов к llm

In [2]:
def conntect_to_llama_by_api(llama_token):
    llama = LlamaAPI(llama_token)
    return llama

def get_answer_by_api(context, llm, system_prompt, question, max_tokens=4096, temperature=0.1, quary=''):
    api_request_json = {
        'model': 'llama3.1-70b',
        'max_tokens': max_tokens,
        'temperature': temperature,
        "messages": [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question.format(context=context, quary=quary)}
        ]
    }
    response = llm.run(api_request_json)
    return response.json()['choices'][0]['message']['content']

### Предобработка данных

In [3]:
data_path = 'hmao_npa.txt'  # path to dataset

with open(data_path, encoding='utf8') as file:
    txt = file.read()

list_docs = txt.split('\n')
list_docs = [doc for doc in list_docs if doc]

In [4]:
new_docs = []
numbers = []
for i in range(len(list_docs)):
    list_docs[i] = list_docs[i].replace('¦', '|')
    ids = list_docs[i].find('|')

    if ids == -1:
        new_docs.append(list_docs[i])
    else:
        new_docs.append(list_docs[i][:ids])
        numbers.append(i)

### Создание эмбеддингов

#### multilingual-e5-base и BM25 

In [4]:
model_name = 'intfloat/multilingual-e5-base'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

sparse_embedding_func = BM25SparseEmbedding(corpus=new_docs)

/home/jovyan/Lacmus/lacmus_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [8]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)

entities = []
i = 0
for doc in tqdm(new_docs):
    for j, text in enumerate(child_splitter.split_text(doc)):
        entity = {
            'doc_id': i,
            'doc_part': j,
            'dense_vector': embeddings.embed_documents([text])[0],
            'sparse_vector': sparse_embedding_func.embed_documents([text])[0],
            'text': text,
        }
        entities.append(entity)
    i += 1
    
with open('embedings_e5.pkl', 'wb') as f:
    pkl.dump(entities, f)

100%|██████████| 1856/1856 [05:38<00:00,  5.48it/s]


#### BGE и BM25

In [50]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction = "search_query:",
    embed_instruction = "search_document:"
)
sparse_embedding_func = BM25SparseEmbedding(corpus=new_docs)

In [11]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
entities_2 = []
i = 0
for doc in tqdm(new_docs):
    for j, text in enumerate(child_splitter.split_text(doc)):
        entity = {
            'doc_id': i,
            'doc_part': j,
            'dense_vector': embeddings.embed_documents([text])[0],
            'sparse_vector': sparse_embedding_func.embed_documents([text])[0],
            'text': text,
        }
        entities_2.append(entity)
    i += 1
with open('embedings_bge.pkl', 'wb') as f:
    pkl.dump(entities_2, f)

100%|██████████| 1856/1856 [07:00<00:00,  4.42it/s]


### Создание колекции 

Два варианта эмбедингов. Нужно выбрать один из них. И в зависимости отвыбора подгрузить эмбединги

In [5]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction = "search_query:",
    embed_instruction = "search_document:"
)
sparse_embedding_func = BM25SparseEmbedding(corpus=new_docs)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mariya.kuznetsova\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
# model_name = 'intfloat/multilingual-e5-base'
# model_kwargs = {'device': 'cuda'}
# encode_kwargs = {'normalize_embeddings': True}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

# sparse_embedding_func = BM25SparseEmbedding(corpus=new_docs)

In [6]:
vector_dim = len(embeddings.embed_documents(['Пара пара пам'])[0])

In [7]:
# connections.connect(uri="./milvus_demo.db")
connections.connect(uri="http://localhost:19530")

In [37]:
connections.connect(uri="http://localhost:19530")
# connections.connect(uri="./milvus_demo.db")
collection_name = 'NPA_'

fields = [
    FieldSchema(
        name='id',
        dtype=DataType.VARCHAR,
        is_primary=True,
        auto_id=True,
        max_length=100,
    ),
    FieldSchema(name='doc_id', dtype=DataType.INT32, max_length=100),
    FieldSchema(name='doc_part', dtype=DataType.INT32, max_length=100),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=vector_dim),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2000),
]

schema = CollectionSchema(fields=fields, enable_dynamic_field=False)
collection = Collection(
    name=collection_name, schema=schema, consistency_level="Strong"
)

dense_index = {"index_type": "FLAT", "metric_type": "IP"}
collection.create_index("dense_vector", dense_index)
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
collection.create_index("sparse_vector", sparse_index)
collection.flush()

In [38]:
with open('embedings_bge.pkl', 'rb') as f:
    entities = pkl.load(f)

# with open('embedings_e5.pkl', 'rb') as f:
#     entities = pkl.load(f)

collection.insert(entities[:8000])
collection.load()
collection.insert(entities[8000:16000])
collection.load()
collection.insert(entities[16000:])
collection.load()

### Создание ретривера

Вот тут нужно с весами поэкспериментировать. Кстати веса [0, 1] и [1, 0] это по сути будет поиск одним ретривером.

In [39]:
sparse_search_params = {"metric_type": "IP"}
dense_search_params = {"metric_type": "IP", "params": {}}
retriever = MilvusCollectionHybridSearchRetriever(
    collection=collection,
    rerank=WeightedRanker(0.5, 0.5),
    anns_fields=['dense_vector', 'sparse_vector'],
    field_embeddings=[embeddings, sparse_embedding_func],
    field_search_params=[dense_search_params, sparse_search_params],
    top_k=5,
    text_field='text',
)

### Переформулировка запроса

In [65]:
question_3 = """ 
    Ты разговариваешь только на русском языке. Ты юрист. Определи чего хочет клиент. Сформулируй его вопрос на юридическом языке.
    Твой ответ должен содержать только сам переформулированный вопрос.
    Вопрос клиента: {context}
"""

question_2 = """ 
    Ты разговариваешь только на русском языке. Ответь на вопрос, опираясь на только нормативно-правовые акты из контектса.
    {quary} 
    
    Нормативные акты: {context}
"""

system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать по 100$.
"""

quary = 'Зачем менять постановление о Департаменте внутренней политики?'

llm = conntect_to_llama_by_api(token)

quary = get_answer_by_api(quary, llm, system_prompt, question_3, max_tokens=4096, temperature=0.5)
quary

'В каких случаях и по каким основаниям возможно изменение постановления о Департаменте внутренней политики в соответствии с действующим законодательством?'

Ищем нужные документв и передаём llm

In [66]:
docs = retriever.invoke(quary)
context = '\n'.join([doc.page_content for doc in docs])

In [67]:
get_answer_by_api(context, llm, system_prompt, question_2, max_tokens=4096, temperature=0.5, quary=quary)

'Изменение постановления о Департаменте внутренней политики в соответствии с действующим законодательством возможно в следующих случаях:\n\n1. Внесение изменений в штатное расписание Департамента внутренней политики в соответствии с трудовым законодательством Российской Федерации и законодательством о государственной гражданской службе (пункт 7.1).\n2. Перераспределение функций и полномочий, возглавляемых органов государственной власти Ханты-Мансийского автономного округа - Югры (пункт 7.2).\n3. Внесение изменений в правовые акты Департамента внутренней политики в соответствии с постановлением Правительства Ханты-Мансийского автономного округа - Югры (пункт 8).\n4. Внесение изменений в постановление о Департаменте внутренней политики в соответствии с решениями Общественного совета при Департаменте внутренней политики Ханты-Мансийского автономного округа - Югры (протокол заседания от 9 декабря 2020 года N 17).\n5. Внесение изменений в постановление о Департаменте внутренней политики в с

In [70]:
get_answer_by_api(context, llm, system_prompt, question_2, max_tokens=4096, temperature=0.5, quary=quary)

'Изменение постановления о Департаменте внутренней политики возможно в соответствии с действующим законодательством в следующих случаях и по следующим основаниям:\n\n1. В соответствии с пунктом 4 статьи 17 Федерального закона от 6 октября 1999 года N 184-ФЗ "Об общих принципах организации законодательных (представительных) и исполнительных органов государственной власти субъектов Российской Федерации", подпунктом 13 пункта 1 и пунктом 4 статьи 34 Устава (Основного закона) Ханты-Мансийского автономного округа - Югры, постановление о Департаменте внутренней политики может быть изменено по решению Губернатора Ханты-Мансийского автономного округа - Югры.\n\n2. В соответствии с постановлением Правительства Ханты-Мансийского автономного округа - Югры от 27 июля 2018 года N 226-п "О модельной государственной программе Ханты-Мансийского автономного округа - Югры, порядке принятия решения о разработке государственных программ Ханты-Мансийского автономного округа - Югры, их формирования, утвержд

Это если нужно колекцию дропнуть

In [36]:
collection.drop()

### Подтянем соседние кусочки и начало

In [68]:
def neighbors(collection, doc_id, part_id):

    # Поиск всех частей документа по doc_id
    # Здесь предполагается, что поле order указывает на порядок частей документа
    parts = collection.query(expr=f"doc_id == {doc_id}", output_fields=["doc_part", "text"])

    # Сортируем части документа по порядку
    sorted_parts = sorted(parts, key=lambda x: x["doc_part"])

    document_start = sorted_parts[0]["text"]
        
    # Извлекаем предыдущую и следующую части (если они существуют)
    previous_part = sorted_parts[part_id - 1]["text"] if part_id > 0 else None
    current_part = sorted_parts[part_id]["text"]
    next_part = sorted_parts[part_id + 1]["text"] if part_id < len(sorted_parts) - 1 else None
    
    # Возвращаем список частей
    return [document_start, previous_part, current_part, next_part]

Второй вариант как создать контекст с дополнительными куосчками

In [69]:
context = ''
for doc in docs:
    other = neighbors(collection, doc.metadata['doc_id'], doc.metadata['doc_part'])
    other = [x for x in other if x is not None]
    other = ' '.join(other)
    context += other + '\n\n'

In [58]:
neighbors(collection, docs[0].metadata['doc_id'], docs[0].metadata['doc_part'])

[{'doc_part': 0, 'text': 'ЗАКОН  ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ от 20.07.2007 № 108-оз.  О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ "О НОРМАТИВНЫХ ПРАВОВЫХ АКТАХ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ" ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ "О НОРМАТИВНЫХ ПРАВОВЫХ АКТАХ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ" Принят Думой Ханты-Мансийского автономного округа - Югры 12 июля 2007 года Статья 1. Внести в Закон Ханты-Мансийского автономного округа - Югры от 25 февраля 2003 года N 14-оз "О нормативных правовых актах Ханты-Мансийского автономного округа - Югры" (с изменениями, внесенными Законами Ханты-Мансийского автономного округа - Югры от 31 декабря 2004 года N 105-оз, 5 июля 2005 года N 55-оз, 26 февраля 2007 года N 7-оз) (Собрание законодательства Ханты-Мансийского автономного округа, 2003, N 2, ст. 103; Собрание законодательства Ханты-Мансийского автон

['ЗАКОН  ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ от 20.07.2007 № 108-оз.  О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ "О НОРМАТИВНЫХ ПРАВОВЫХ АКТАХ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ" ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ЗАКОН ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ "О НОРМАТИВНЫХ ПРАВОВЫХ АКТАХ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ" Принят Думой Ханты-Мансийского автономного округа - Югры 12 июля 2007 года Статья 1. Внести в Закон Ханты-Мансийского автономного округа - Югры от 25 февраля 2003 года N 14-оз "О нормативных правовых актах Ханты-Мансийского автономного округа - Югры" (с изменениями, внесенными Законами Ханты-Мансийского автономного округа - Югры от 31 декабря 2004 года N 105-оз, 5 июля 2005 года N 55-оз, 26 февраля 2007 года N 7-оз) (Собрание законодательства Ханты-Мансийского автономного округа, 2003, N 2, ст. 103; Собрание законодательства Ханты-Мансийского автономного округа - Югры,',
